In [2]:
# -*- coding: utf-8 -*-
"""
Simulated Annealing para el Diet Problem
Minimizar el coste cumpliendo restricciones nutricionales
"""

from __future__ import print_function
import random
import csv
from simanneal import Annealer

# ==========================
# CONFIGURACIÓN DEL MODELO
# ==========================

# Rangos de nutrientes semanales (los buenos)
NUTRIENT_BOUNDS = {
    "Calories":      (14000.0, 17500.0),
    "Cholesterol":   (0.0,    2100.0),
    "Total_Fat":     (300.0,  650.0),
    "Sodium":        (0.0,    16000.0),
    "Carbohydrates": (1500.0, 2600.0),
    "Dietary_Fiber": (140.0,  280.0),
    "Protein":       (350.0,  700.0),
    "Vit_A":         (5000.0, 20000.0),
    "Vit_C":         (500.0,  7000.0),
    "Calcium":       (4900.0, 10500.0),
    "Iron":          (60.0,   280.0),
}

# Clasificación fija de los alimentos (mismo dataset pequeño)
CATEGORY_MAP = {
    "Frozen Broccoli":        "veg",
    "Carrots Raw":            "veg",
    "Celery Raw":             "veg",
    "Frozen Corn":            "veg",
    "Lettuce Iceberg Raw":    "veg",
    "Peppers Sweet Raw":      "veg",
    "Tomato Red Ripe Raw":    "veg",

    "Potatoes Baked":         "carb",
    "Bagels":                 "carb",
    "Wheat Bread":            "carb",
    "White Bread":            "carb",
    "Cap'N Crunch":           "carb",
    "Cheerios":               "carb",
    "Corn Flakes Kellogg'S":  "carb",
    "Raisin Bran Kellogg'S":  "carb",
    "Rice Krispies":          "carb",
    "Special K":              "carb",
    "Oatmeal":                "carb",
    "Couscous":               "carb",
    "Macaroni cooked":        "carb",
    "White Rice":             "carb",

    "Apple Raw w/Skin":       "fruit",
    "Banana":                 "fruit",
    "Grapes":                 "fruit",
    "Kiwifruit Raw Fresh":    "fruit",
    "Oranges":                "fruit",

    "Tofu":                   "legume",
    "Peanut Butter":          "legume",

    "Butter Regular":         "dairy",
    "Cheddar Cheese":         "dairy",
    "Whole Milk":             "dairy",
    "Lowfat Milk":            "dairy",
    "Skim Milk":              "dairy",

    "Roasted Chicken":        "meat",
    "Poached Eggs":           "meat",
    "Scrambled Eggs":         "meat",
    "Bologna Turkey":         "meat",
    "Frankfurter Beef":       "meat",
    "Ham Sliced Extralean":   "meat",
    "Kielbasa Pork":          "meat",
    "Pork":                   "meat",

    "Sardines in Oil":        "fish",
    "White Tuna in Water":    "fish",

    "Pizza w/Pepperoni":      "prepared",
    "Hamburger w/Toppings":   "prepared",
    "Hotdog Plain":           "prepared",
    "Spaghetti W/ Sauce":     "prepared",

    "Oatmeal Cookies":        "sweet",
    "Apple Pie":              "sweet",
    "Chocolate Chip Cookies": "sweet",
    "Malt-O-Meal Choc":       "sweet",

    "Popcorn Air-Popped":     "snack",
    "Potato Chips":           "snack",
    "Pretzels":               "snack",
    "Tortilla Chips":         "snack",

    "Chicken Noodle Soup":    "soup",
    "Splt Pea&Ham Soup":      "soup",
    "Veggie Beef Soup":       "soup",
    "New Eng Clam Chwd":      "soup",
    "Tomato Soup":            "soup",
    "Crm Mshrm Soup":         "soup",
    "Bean Bacon Soup":        "soup",
}

# Bounds semanales por categoría (ajustados)
CATEGORY_BOUNDS = {
    "veg":     (14, 28),   # 2–4 raciones de verdura/día
    "fruit":   (7, 21),    # 1–3 frutas/día
    "carb":    (7, 21),    # 1.5–4 raciones cereales/tubérculos/día
    "legume":  (2, 5),     # 2–5 raciones tofu+peanut butter/sem
    "dairy":   (5, 11),    # 3–10 lácteos/sem
    "meat":    (3, 7),     # proteína animal suficiente sin exceso
    "fish":    (3, 7),     # 2–7 pescados/sem

    "prepared": (0, 2),    # pizza, hamburguesa, hotdog, spaghetti w/sauce
    "snack":    (0, 2),    # snacks salados
    "sweet":    (0, 3),    # dulces/cereales azucarados
    "soup":     (0, 5),    # sopas procesadas
}

# Precios nuevos (con ligera aleatoriedad, ya calculados)
price_map = {
    "Lowfat Milk": 0.49,
    "Whole Milk": 0.44,
    "Apple Pie": 0.63,
    "Apple Raw w/Skin": 0.53,
    "Bagels": 0.37,
    "Banana": 0.42,
    "Bean Bacon Soup": 1.58,
    "Bologna Turkey": 0.52,
    "Butter Regular": 0.33,
    "Cap'N Crunch": 0.47,
    "Carrots Raw": 0.33,
    "Celery Raw": 0.28,
    "Cheddar Cheese": 0.86,
    "Cheerios": 0.42,
    "Chicken Noodle Soup": 1.63,
    "Chocolate Chip Cookies": 0.47,
    "Corn Flakes Kellogg'S": 0.43,
    "Couscous": 0.63,
    "Crm Mshrm Soup": 1.53,
    "Frankfurter Beef": 0.88,
    "Frozen Broccoli": 0.91,
    "Frozen Corn": 0.47,
    "Grapes": 0.53,
    "Ham Sliced Extralean": 0.63,
    "Hamburger w/Toppings": 4.17,
    "Hotdog Plain": 2.07,
    "Kielbasa Pork": 0.52,
    "Kiwifruit Raw Fresh": 0.52,
    "Lettuce Iceberg Raw": 0.32,
    "Macaroni cooked": 0.64,
    "Malt-O-Meal Choc": 1.01,
    "New Eng Clam Chwd": 1.57,
    "Oatmeal": 1.03,
    "Oatmeal Cookies": 0.42,
    "Oranges": 0.42,
    "Peanut Butter": 0.47,
    "Peppers Sweet Raw": 0.71,
    "Pizza w/Pepperoni": 2.05,
    "Poached Eggs": 0.38,
    "Popcorn Air-Popped": 0.38,
    "Pork": 1.38,
    "Potato Chips": 0.64,
    "Potatoes Baked": 0.48,
    "Pretzels": 0.52,
    "Raisin Bran Kellogg'S": 0.59,
    "Rice Krispies": 0.48,
    "Roasted Chicken": 3.82,
    "Sardines in Oil": 1.15,
    "Scrambled Eggs": 0.37,
    "Skim Milk": 0.48,
    "Spaghetti W/ Sauce": 2.54,
    "Special K": 0.48,
    "Splt Pea&Ham Soup": 1.63,
    "Tofu": 0.92,
    "Tomato Red Ripe Raw": 0.37,
    "Tomato Soup": 1.27,
    "Tortilla Chips": 0.63,
    "Veggie Beef Soup": 1.56,
    "Wheat Bread": 0.38,
    "White Bread": 0.33,
    "White Rice": 0.57,
    "White Tuna in Water": 1.77,
}

LAMBDA = 100.0  # peso de la penalización vs coste


class DietProblem(Annealer):
    """
    Problema de la dieta utilizando Recocido Simulado (SA).
    state: cantidades (porciones) de cada alimento
    foods: lista de alimentos con precio, nutrientes y categoría
    """

    def __init__(self, state, foods, nutrient_bounds, max_portion=10.0):
        self.foods = foods
        self.nutrient_bounds = nutrient_bounds
        self.nutrient_names = list(nutrient_bounds.keys())
        self.max_portion = max_portion
        super(DietProblem, self).__init__(state)

    def move(self):
        """Genera un estado vecino: modifica en ±1 la cantidad de 1 alimento (valores enteros)."""
        i = random.randint(0, len(self.state) - 1)

        # Cambio entero: -1 o +1
        delta = random.choice([-1, 1])

        new_val = self.state[i] + delta

        # Forzar al rango [0, max_portion]
        if new_val < 0:
            new_val = 0
        elif new_val > self.max_portion:
            new_val = self.max_portion

        # Asegurar entero
        self.state[i] = int(new_val)

    def energy(self):
        """
        Función objetivo:
        coste + LAMBDA * (penalización por nutrientes + penalización por categorías)
        """
        cost = 0.0
        totals = {nut: 0.0 for nut in self.nutrient_names}
        cat_totals = {cat: 0.0 for cat in CATEGORY_BOUNDS.keys()}

        for qty, food in zip(self.state, self.foods):
            if qty <= 0:
                continue
            cost += qty * food["Price"]
            for nut in self.nutrient_names:
                totals[nut] += qty * food[nut]

            cat = food["Category"]
            if cat in cat_totals:
                cat_totals[cat] += qty

        # Penalización nutrientes (lineal, como en el GA)
        nut_penalty = 0.0
        for nut, (mn, mx) in self.nutrient_bounds.items():
            v = totals[nut]
            if v < mn:
                nut_penalty += (mn - v)
            elif v > mx:
                nut_penalty += (v - mx)

        # Penalización por patrón de dieta (categorías)
        cat_penalty = 0.0
        for cat, (mn, mx) in CATEGORY_BOUNDS.items():
            v = cat_totals[cat]
            if v < mn:
                cat_penalty += (mn - v)
            elif v > mx:
                cat_penalty += (v - mx)

        penalty = nut_penalty + cat_penalty

        return cost + LAMBDA * penalty


def load_foods(csv_file, nutrient_bounds):
    """Lee el CSV y devuelve una lista de alimentos con los nutrientes relevantes + categoría + precio nuevo."""
    foods = []
    with open(csv_file, newline='', encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["Name"]
            food = {
                "Name": name,
                # sobreescribimos Price con nuestro price_map “realista”
                "Price": float(price_map[name]),
                "Category": CATEGORY_MAP[name],
            }
            for nut in nutrient_bounds.keys():
                food[nut] = float(row[nut])
            foods.append(food)
    return foods


if __name__ == '__main__':
    random.seed(10)

    nutrient_bounds = NUTRIENT_BOUNDS

    # === Cargar alimentos del CSV ===
    foods = load_foods("diet_with_prices.csv", nutrient_bounds)
    n_foods = len(foods)
    print(f"{n_foods} alimentos cargados.")

    # === Máximo de porciones para todos los alimentos ===
    MAX_PORTION = 7

    # === Estado inicial aleatorio (enteros) ===
    init_state = [random.randint(0, MAX_PORTION) for _ in range(n_foods)]

    # === Ejecutar Simulated Annealing ===
    diet = DietProblem(init_state, foods, nutrient_bounds, max_portion=MAX_PORTION)
    diet.steps = 200000            # iteraciones totales
    diet.Tmax = 1000.0             # temperatura inicial
    diet.Tmin = 1e-3               # temperatura final
    diet.copy_strategy = "slice"

    state, energy = diet.anneal()

    # === Resultados ===
    print("\n📌 Mejor coste penalizado encontrado:", energy)

    # Calcular coste real y totales de nutrientes
    cost = 0.0
    totals = {nut: 0.0 for nut in nutrient_bounds.keys()}
    cat_totals = {cat: 0.0 for cat in CATEGORY_BOUNDS.keys()}

    for qty, food in zip(state, foods):
        if qty <= 0:
            continue
        cost += qty * food["Price"]
        for nut in nutrient_bounds.keys():
            totals[nut] += qty * food[nut]
        cat_totals[food["Category"]] += qty

    print("\n💰 Coste real final (sin penalización):", cost)
    print("\n🔍 Nutrientes totales obtenidos:")
    for nut, (mn, mx) in nutrient_bounds.items():
        print(f"  {nut}: {totals[nut]:.2f}  (mín {mn} – máx {mx})")

    print("\n🥗 Raciones por categoría:")
    for cat, (mn, mx) in CATEGORY_BOUNDS.items():
        print(f"  {cat}: {cat_totals[cat]:.2f}  (mín {mn} – máx {mx})")

    print("\n🥗 Alimentos incluídos en la dieta:")
    for qty, food in zip(state, foods):
        if qty > 0:
            print(f"  {food['Name']}: {qty} porciones")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
   758.57758       3794.90    71.80%    25.75%     0:00:00     0:00:06

62 alimentos cargados.


     0.00100         53.68    31.05%     0.00%     0:00:06     0:00:00


📌 Mejor coste penalizado encontrado: 53.64

💰 Coste real final (sin penalización): 53.64

🔍 Nutrientes totales obtenidos:
  Calories: 14003.70  (mín 14000.0 – máx 17500.0)
  Cholesterol: 1205.00  (mín 0.0 – máx 2100.0)
  Total_Fat: 622.80  (mín 300.0 – máx 650.0)
  Sodium: 9521.70  (mín 0.0 – máx 16000.0)
  Carbohydrates: 1770.90  (mín 1500.0 – máx 2600.0)
  Dietary_Fiber: 149.90  (mín 140.0 – máx 280.0)
  Protein: 410.80  (mín 350.0 – máx 700.0)
  Vit_A: 18742.80  (mín 5000.0 – máx 20000.0)
  Vit_C: 923.30  (mín 500.0 – máx 7000.0)
  Calcium: 5123.40  (mín 4900.0 – máx 10500.0)
  Iron: 245.50  (mín 60.0 – máx 280.0)

🥗 Raciones por categoría:
  veg: 14.00  (mín 14 – máx 28)
  fruit: 20.00  (mín 7 – máx 21)
  carb: 21.00  (mín 7 – máx 21)
  legume: 5.00  (mín 2 – máx 5)
  dairy: 11.00  (mín 5 – máx 11)
  meat: 7.00  (mín 3 – máx 7)
  fish: 3.00  (mín 3 – máx 7)
  prepared: 2.00  (mín 0 – máx 2)
  snack: 2.00  (mín 0 – máx 2)
  sweet: 3.00  (mín 0 – máx 3)
  soup: 1.00  (mín 0 – máx 5)